In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from satellite_and_aeronet import calitoo, aeronet
from IPython.display import display 

filenames = [r'../daten calitoo/preprocessed_data_with_AOD_all.csv']
# = [r'../daten calitoo/0123_20240604_075512_10_ours.txt', r'../daten calitoo/0425_20240604_082717_10.txt', r'../daten calitoo/0427_20240604_082717_10.txt']
columns_with_commas = ['Elevation', 'AOT465', 'AOT540', 'AOT619', 'Alpha', 'R²']
columns_to_numbers = ['Temperature', 'Pression', 'RAW465', 'RAW540', 'RAW619','Elevation', 'AOT465', 'AOT540', 'AOT619', 'Alpha', 'R²']

locations = ['laegern', 'payerne']
coord_threshold = [4700,800,4770,870] # lat0, long0, lat1, long1
coord_threshold = [4500, 8, 4900, 1000]
alpha_threshold = 2
r_threshold = 0.8
aot_threshold = 0.8

def plot_calitoo(dataframe):
    wavelengths_cal = ['AOT465'] #, 'AOT540', 'AOT619'
    for wavelength in wavelengths_cal:
        plt.scatter(dataframe['Date'].dt.dayofyear, dataframe[wavelength].where(dataframe[wavelength]<0.5), label=f'{wavelength} cal')
    #plt.scatter(dataframe['Date'].dt.dayofyear, np.full((dataframe.shape[0],), 0.02), label='Datapoints cal')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xlabel('Day of year')
    plt.ylabel('AOD')
    plt.title('Calitoo')
    plt.ylim([0,0.6])


for index, filename in enumerate(filenames):
    data = pd.read_csv(filenames[index], skiprows=0, delimiter=',')
    columns_with_commas = ['Elevation', 'AOT465', 'AOT540', 'AOT619', 'Alpha', 'R²']
    display(data)
    if index == 0:
        for column in columns_with_commas:
            data[column] = pd.to_numeric(data[column].str.replace(',', '.'), errors='coerce')
            
    data = data.replace('-.--', np.nan)

    for column in columns_to_numbers:
        data[column] = pd.to_numeric(data[column])
    data['Latitude'] = pd.to_numeric(data['Latitude'].str.replace('N', ''), errors='coerce')
    data['Longitude'] = pd.to_numeric(data['Longitude'].str.replace('E', ''), errors='coerce')
    data['Altitude'] = pd.to_numeric(data['Altitude'].str.replace('E', ''), errors='coerce')
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
    data['Time'] = data['Time'].str.replace('.', '0')
    data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S',  errors='coerce').dt.time
    
    filtered_data = data[
        # Latitude condition (between lat0 and lat1)
        (data['Latitude'] >= coord_threshold[0]) & 
        (data['Latitude'] <= coord_threshold[2]) & 
        
        # Longitude condition (between long0 and long1)
        (data['Longitude'] >= coord_threshold[1]) & 
        (data['Longitude'] <= coord_threshold[3]) & 
        
        # Alpha threshold condition 
        (data['Alpha'] <= alpha_threshold) & 
        
        # R² threshold condition
        (data['R²'] >= r_threshold)&

        (data['AOT465']< aot_threshold)&
        (data['AOT540']< aot_threshold)&
        (data['AOT540']< aot_threshold)
    ]
    print(data.isna().sum().sum())



    plot_calitoo(filtered_data)
    #display(filtered_data)

aeronet(locations[1])


,Unnamed: 0.1,Unnamed: 0,Date,Time,Temperature,Pression,RAW465,RAW540,RAW619,Altitude,...,E0,precalibrated_AOT465,precalibrated_AOT540,precalibrated_AOT619,tau_aerosols_465_calitoo,tau_aerosols_540_calitoo,tau_aerosols_619_calitoo,tau_aerosols_465_expected,tau_aerosols_540_expected,tau_aerosols_619_expected
0,0,0,2024-06-04,07:55:12,21,841,2607.552239,2912.505010,2225.331030,0000M,...,0.970642,0.5173,0.4769,0.4104,0.012054,0.022512,0.029485,0.003896,0.016013,0.019467
1,1,1,2024-06-04,07:55:25,22,841,2595.189289,2902.202552,2221.210047,01608,...,0.970642,0.5210,0.4797,0.4120,0.015295,0.024928,0.030749,0.007137,0.018430,0.020731
2,2,2,2024-06-04,09:00:01,23,841,2721.909528,3004.196891,2278.903814,01608,...,0.970642,0.6027,0.5540,0.4757,0.013987,0.022542,0.029326,0.004396,0.014902,0.017548
3,3,3,2024-06-04,09:00:22,23,841,2713.667562,2980.501236,2290.236518,01614,...,0.970642,0.6057,0.5608,0.4721,0.016693,0.029084,0.025483,0.007089,0.021434,0.013690
4,4,4,2024-06-04,09:00:36,23,841,2733.242233,3014.499349,2277.873568,01615,...,0.970642,0.6003,0.5520,0.4767,0.010923,0.019978,0.029828,0.001319,0.012328,0.018036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,347,390,2024-12-01,13:22:47,18,981,1178.619794,1669.873184,1498.031583,00407,...,1.030018,0.0935,0.0806,0.0661,0.087308,0.077282,0.064155,0.084232,0.074832,0.060378
348,348,391,2024-12-01,13:22:55,18,981,1175.707224,1667.931471,1487.352161,00408,...,1.030018,0.0939,0.0808,0.0678,0.087945,0.077581,0.065995,0.084869,0.075131,0.062218
349,349,392,2024-12-01,13:49:30,20,965,1134.931251,1658.222906,1505.798435,00574,...,1.030018,0.0574,0.0509,0.0422,0.048887,0.046394,0.039571,0.046319,0.044347,0.036417
350,350,393,2024-12-01,13:49:38,20,965,1139.785534,1657.252049,1513.565287,00580,...,1.030018,0.0562,0.0509,0.0409,0.046043,0.045204,0.037540,0.043494,0.043174,0.034411


AttributeError: Can only use .str accessor with string values!

In [2]:
file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a subdirectory)
        if os.path.isfile(file_path):
            print(filename)

IndentationError: unexpected indent (2592306274.py, line 4)

In [ ]:
data